In [1]:
# For following a second Drone
#Must have second drone in sights of Mambo Drone
import numpy as np
import matplotlib.pyplot as plt
import glob
import cv2
import random
import tensorflow as tf



In [2]:
import pickle
import zipfile 

In [3]:
training_file = ".traffic-signs-data/train.p"
validation_file= ".traffic-signs-data/valid.p"
testing_file = ".traffic-signs-data/valid.p"

with open(training_file, mode = 'rb') as f:
    train = pickle.load(f)

with open(testing_file, mode = 'rb') as f:
    test = pickle.load(f)
    
with open(validation_file, mode = 'rb') as f:
    valid = pickle.load(f)
    

X_train, y_train = train['features'], train['labels']
